In [33]:
import boto3
from botocore.config import Config
import sys
import os
import time
import requests
from bs4 import BeautifulSoup


In [34]:
ohio_region  = Config(region_name="us-east-2")
north_virginia_region  = Config(region_name="us-east-1")

In [35]:
ohio_ec2 = boto3.resource('ec2', config=ohio_region)
north_virginia_ec2 = boto3.resource('ec2', config=north_virginia_region)

In [36]:
script_postgres = '''#!/bin/bash
sudo apt update
sudo apt install postgresql postgresql-contrib -y
sudo -i -u postgres bash << EOF
createuser -s cloud -W
cloud
createdb -O cloud tasks
echo "listen_addresses = '*'" >>  /etc/postgresql/12/main/postgresql.conf
echo "host all all 0.0.0.0/0 trust" >> /etc/postgresql/12/main/pg_hba.conf
EOF
sudo ufw allow 5432/tcp
sudo systemctl restart postgresql'''



In [37]:
script_mysql = '''#!/bin/bash
sudo apt update
sudo apt install mysql-server -y
sudo mysql -uroot -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'cloud';CREATE USER 'root'@'%' IDENTIFIED BY 'cloud';GRANT ALL PRIVILEGES ON *.* TO 'root'@'%';FLUSH PRIVILEGES;CREATE DATABASE tasks;"
sudo sed -i 's/bind-address/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo sed -i 's/mysqlx/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo ufw allow 3306/tcp
sudo systemctl restart mysql
'''


In [38]:
security_group_iterator = ohio_ec2.security_groups.all()

In [40]:
security_group = ohio_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup2',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)

security_group2 = north_virginia_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup2',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group2.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)


{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-07cd720760aac2091',
   'GroupId': 'sg-0b642c6ada285968b',
   'GroupOwnerId': '903616414837',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 0,
   'ToPort': 10000,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': 'a9743144-8fdd-4d2e-a003-a6aa0669f2e7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a9743144-8fdd-4d2e-a003-a6aa0669f2e7',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '721',
   'date': 'Wed, 17 Nov 2021 16:39:06 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [41]:
instance_ohio = ohio_ec2.create_instances(
        ImageId="ami-00399ec92321828f5",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group.group_id],
        UserData = script_mysql
    )



In [42]:
instance_ohio[0].wait_until_running()
instance_ohio[0].reload()
public_ip_ohio = instance_ohio[0].public_ip_address
time.sleep(60)

In [43]:
print(public_ip_ohio)

3.144.173.4


In [51]:
script_django_mysql = f"""#!/bin/bash
cd /home/ubuntu
sudo apt update
sudo apt-get install python3-dev default-libmysqlclient-dev build-essential -y
git clone https://github.com/raulikeda/tasks.git
cd tasks
sudo sed -i 's/node1/{public_ip_ohio}/g' portfolio/settings.py
sudo sed -i 's/django.db.backends.postgresql/django.db.backends.mysql/g' portfolio/settings.py
sudo sed -i 's/5432/3306/g' portfolio/settings.py
sudo sed -i "s/'USER': 'cloud'/'USER': 'root'/g" portfolio/settings.py
echo "mysqlclient" >> requirements.txt
./install.sh
sudo reboot 
"""

In [52]:
north_virginia = north_virginia_ec2.create_instances(
        ImageId="ami-09e67e426f25ce0d7",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group2.group_id],
        UserData = script_django_mysql
    )

In [54]:
north_virginia[0].wait_until_running()
north_virginia[0].reload()
#time.sleep(60)

In [55]:
django_img = north_virginia[0].create_image(Name='django_img3')


In [56]:
print(django_img.state)
django_img.reload()

pending


In [57]:
while django_img.state != 'available':
    django_img.reload()
north_virginia[0].terminate()

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-0b9a70e208a0a29b6',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': 'c5bcbb80-010a-489b-9fea-c3639a8e76db',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c5bcbb80-010a-489b-9fea-c3639a8e76db',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Wed, 17 Nov 2021 17:00:34 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [58]:
print(django_img.state)


available


In [59]:
autoscaling  = boto3.client('autoscaling', config=north_virginia_region)
elb = boto3.client('elbv2', config=north_virginia_region)


In [106]:
create_launch_response = autoscaling.create_launch_configuration(
    LaunchConfigurationName='LCV999',
    ImageId= django_img.id,
    KeyName='jpgianfaldoni',
    SecurityGroups=[
        security_group2.id,
    ],
    BlockDeviceMappings=[
                        {
                            'DeviceName': '/dev/sda1',
                            'Ebs': {
                                'DeleteOnTermination': True,
                                'VolumeSize': 8,
                                'VolumeType': 'gp2'
                            },
                        },
                ],
    InstanceType='t2.micro',
    
)


In [109]:
create_autoscaling_response = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName='AutoScalingV1',
    LaunchConfigurationName='LCV999',
    MinSize=1,
    MaxSize=3,
    DesiredCapacity=3,
    DefaultCooldown=60,
    AvailabilityZones=[
        'us-east-1a',
        'us-east-1b',
        'us-east-1c',
        'us-east-1d',
        'us-east-1e',
        'us-east-1f',
    ]
)

In [110]:
create_target_response = elb.create_target_group(
    Name='Tgroup5',
    Protocol='HTTP',
    Port=8080,
    VpcId='vpc-ebd11591',
    HealthCheckPath = '/tasks/',
    HealthCheckProtocol='HTTP',
    HealthCheckEnabled=True,
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=15,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=5,
    TargetType='instance',
    IpAddressType='ipv4'
)

In [111]:
tg_arn = create_target_response['TargetGroups'][0]['TargetGroupArn']

In [112]:
create_lb_response = elb.create_load_balancer(
    Name='Elb3',
    Subnets=[
        'subnet-54296d1e',
        'subnet-7ebe0d50',
        'subnet-13fa4b4f',
        'subnet-e0f44387',
    ],

    SecurityGroups=[
        security_group2.id,
    ],
    Type='application',
    IpAddressType='ipv4',
)

In [113]:
lb_arn = create_lb_response['LoadBalancers'][0]['LoadBalancerArn']

In [114]:
ab_response = autoscaling.attach_load_balancer_target_groups(
    AutoScalingGroupName='AutoScalingV1',
    TargetGroupARNs=[
        tg_arn,
    ]
)

In [115]:
listener_response = elb.create_listener(
    LoadBalancerArn= lb_arn,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': tg_arn,
        },
    ],
    Port = 80,
    Protocol = 'HTTP',
)

In [116]:
lb_url = create_lb_response['LoadBalancers'][0]['DNSName']
lb_url

'Elb3-1870221194.us-east-1.elb.amazonaws.com'

In [117]:
URL = f'http://{lb_url}/admin/login/?next=/admin/'

In [118]:
URL

'http://Elb3-1870221194.us-east-1.elb.amazonaws.com/admin/login/?next=/admin/'

In [180]:
client = requests.session()

client.get(URL) 

csrftoken = client.cookies['csrftoken']

    
login_data = dict(username='cloud', password='cloud', csrfmiddlewaretoken=csrftoken, next='/admin/')
r = client.post(URL, data=login_data, headers=dict(Referer=URL))
cookies = dict(sessionid=client.cookies.get('sessionid'))
print(cookies)

{'sessionid': 'h1dkit23upd3pyaslj4h3x9by6jh9isg'}


In [182]:
URL_add = f"http://{lb_url}/admin/auth/user/add/"


In [183]:
def add_user(URL, username, password1, password2,cookies):
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken, username=username, password1=password1,
                     password2 =password2, _save = "Save")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2.text
    

In [184]:
add_user(URL_add, 'TESTECLOUD', 'Banana888', 'Banana888', cookies)

'<!DOCTYPE html>\n\n<html lang="en-us" dir="ltr">\n<head>\n<title>TESTECLOUD | Change user | Django site admin</title>\n<link rel="stylesheet" type="text/css" href="/static/admin/css/base.css">\n\n  <link rel="stylesheet" type="text/css" href="/static/admin/css/nav_sidebar.css">\n  <script src="/static/admin/js/nav_sidebar.js" defer></script>\n\n<link rel="stylesheet" type="text/css" href="/static/admin/css/forms.css">\n\n\n<script src="/admin/jsi18n/"></script>\n<script src="/static/admin/js/vendor/jquery/jquery.js"></script>\n<script src="/static/admin/js/calendar.js"></script>\n<script src="/static/admin/js/jquery.init.js"></script>\n<script src="/static/admin/js/admin/DateTimeShortcuts.js"></script>\n<script src="/static/admin/js/core.js"></script>\n<script src="/static/admin/js/admin/RelatedObjectLookups.js"></script>\n<script src="/static/admin/js/SelectBox.js"></script>\n<script src="/static/admin/js/actions.js"></script>\n<script src="/static/admin/js/SelectFilter2.js"></script

In [176]:
def delete_user(user_id, cookies):
    URL_delete = f"http://{lb_url}/admin/auth/user/{user_id}/delete/"
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken,post = "yes")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2.text

In [179]:
print(delete_user(2, cookies))

<!DOCTYPE html>

<html lang="en-us" dir="ltr">
<head>
<title>Log in | Django site admin</title>
<link rel="stylesheet" type="text/css" href="/static/admin/css/base.css">

  <link rel="stylesheet" type="text/css" href="/static/admin/css/nav_sidebar.css">
  <script src="/static/admin/js/nav_sidebar.js" defer></script>

<link rel="stylesheet" type="text/css" href="/static/admin/css/login.css">





    <meta name="viewport" content="user-scalable=no, width=device-width, initial-scale=1.0, maximum-scale=1.0">
    <link rel="stylesheet" type="text/css" href="/static/admin/css/responsive.css">
    

<meta name="robots" content="NONE,NOARCHIVE">
</head>


<body class=" login"
  data-admin-utc-offset="0">

<!-- Container -->
<div id="container">

    
    <!-- Header -->
    <div id="header">
        <div id="branding">
        
<h1 id="site-name"><a href="/admin/">Django administration</a></h1>

        </div>
        
        
    </div>
    <!-- END Header -->
    
    

    <div class="mai